https://www.tensorflow.org/guide/eager#object_based_saving

In [3]:
import tensorflow as tf
tf.enable_eager_execution()

from TPM_from_VCS.data import toy_data_generator
from TPM_from_VCS.models.etp import etp

In [4]:
class Model(tf.keras.Model):
  def __init__(self):
    super(Model, self).__init__()
    self.W = tf.Variable(tf.random_normal((2, 5), dtype=tf.float64), name='weight')
    self.B = tf.Variable(tf.random_normal((2, 1), dtype=tf.float64), name='bias')

    
  def call(self, inputs):
    return tf.add(tf.matmul(self.W, tf.reshape(inputs, (5, -1))), self.B)

# The loss function to be optimized
def loss(model, inputs, targets):
  ans = etp.get_best_etp(model(inputs), targets)
  return tf.reduce_mean(tf.square(ans))

def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return tape.gradient(loss_value, [model.W, model.B])

# Define:
# 1. A model.
# 2. Derivatives of a loss function with respect to model parameters.
# 3. A strategy for updating the variables based on the derivatives.
model = Model()
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

a = toy_data_generator.create_dataset(1)
training_inputs = a[0][0]
training_outputs = a[0][1]

print("Initial loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))

# Training loop
for j in range(10):
    for i in range(300):
      grads = grad(model, training_inputs, training_outputs)
      optimizer.apply_gradients(zip(grads, [model.W, model.B]),
                                global_step=tf.train.get_or_create_global_step())
      if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, training_inputs, training_outputs)))
        
    a = toy_data_generator.create_dataset(1)
    training_inputs = a[0][0]
    training_outputs = a[0][1]

print("Final loss: {:.3f}".format(loss(model, training_inputs, training_outputs)))
print("W = {}, B = {}".format(model.W.numpy(), model.B.numpy()))

Number of nodes currently : 79
Completed VC processing
DEBUG: A Shape: (2, 1)
DEBUG: B Shape: (2,)
Something is wrong!!



InvalidArgumentError: cannot compute MatMul as input #0(zero-based) was expected to be a float tensor but is a double tensor [Op:MatMul] name: MatMul/

https://www.tensorflow.org/guide/summaries_and_tensorboard   : use tensorboard

https://stackoverflow.com/questions/40050397/deep-learning-nan-loss-reasons